In [ ]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [ ]:
from pathlib import Path

IMAGES_PATH = Path() / "images" / "decision_trees"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Decision Tree

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris(as_frame=True)
X_iris = iris.data[["petal length (cm)", "petal width (cm)"]].values
y_iris = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf.fit(X_iris, y_iris)

In [ ]:
from sklearn.tree import export_graphviz

export_graphviz(
    tree_clf,
    out_file=str(IMAGES_PATH / "iris_tree.dot"),
    feature_names=["petal length (cm)", "petal width (cm)"],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)

In [ ]:
from graphviz import Source

Source.from_file(IMAGES_PATH / "iris_tree.dot")

Gini measures the *Gini impurity* where a node is "pure" (gini = 0) if all the training instances it applies to belong to the same class!

Note that while scikit-learn uses the CART (Classification and Regression Tree) algorithm that produces only *binary tree*, there are other algorihtms (e.g. ID3) that can produce trees with nodes that have MORE than 2 children.

In [ ]:
help(tree_clf.tree_)

# Making Predictions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# extra code – just formatting details
from matplotlib.colors import ListedColormap
custom_cmap = ListedColormap(['#fafab0', '#9898ff', '#a0faa0'])
plt.figure(figsize=(8, 4))

lengths, widths = np.meshgrid(np.linspace(0, 7.2, 100), np.linspace(0, 3, 100))
X_iris_all = np.c_[lengths.ravel(), widths.ravel()]
y_pred = tree_clf.predict(X_iris_all).reshape(lengths.shape)
plt.contourf(lengths, widths, y_pred, alpha=0.3, cmap=custom_cmap)
for idx, (name, style) in enumerate(zip(iris.target_names, ("yo", "bs", "g^"))):
    plt.plot(X_iris[:, 0][y_iris == idx], X_iris[:, 1][y_iris == idx],
             style, label=f"Iris {name}")

# extra code – this section beautifies and saves Figure 6–2
tree_clf_deeper = DecisionTreeClassifier(max_depth=3, random_state=42)
tree_clf_deeper.fit(X_iris, y_iris)
th0, th1, th2a, th2b = tree_clf_deeper.tree_.threshold[[0, 2, 3, 6]]
plt.xlabel("Petal length (cm)")
plt.ylabel("Petal width (cm)")
plt.plot([th0, th0], [0, 3], "k-", linewidth=2)
plt.plot([th0, 7.2], [th1, th1], "k--", linewidth=2)
plt.plot([th2a, th2a], [0, th1], "k:", linewidth=2)
plt.plot([th2b, th2b], [th1, 3], "k:", linewidth=2)
plt.text(th0 - 0.05, 1.0, "Depth=0", horizontalalignment="right", fontsize=15)
plt.text(3.2, th1 + 0.02, "Depth=1", verticalalignment="bottom", fontsize=13)
plt.text(th2a + 0.05, 0.5, "(Depth=2)", fontsize=11)
plt.axis([0, 7.2, 0, 3])
plt.legend()
save_fig("decision_tree_decision_boundaries_plot")

plt.show()

# Estimating Class Probabilities

In [ ]:
tree_clf.predict_proba([[5, 1.5]]).round(3)

In [ ]:
tree_clf.predict([[5, 1.5]])

# Computational Complexity

Traversing a decision tree requires going through roughly O($log_2$(m)) nodes, where $log_2$(m) is the *binary logarithm* of m, equal to log(m) / log(2). Since each nodes only requires the checking of value of one feature, overall prediction complexity is not affected by the number of features and hence stays O($log_2$(m)).


Though for the training algorithm, since the algorithm will compare all features on all samples at each node, this will result in a training complexity of O(n x m $log_2$(m))

By default, the DecisionTreeClassifier uses the Gini impurity measure, but the *entropy* impurity measure can also be used by setting the criterion hyperparameter to "entropy" (comes from thermodynamics where entropy approaches 0 when molecules are still and well ordered). In information theory, it measures the average information content of a message. When it is 0, all messages are identical.

In machine learning, a set's entropy is - when it contains instances of only 1 class.

In general, it doesn't really matter whether we use Gini or Entropy, but note that:
- Gini impurity is slightly faster to compute
- When differing, Gini tends to isolate the most frequent class in its own branch of the tree, while entropy tends to produce slightly more balanced trees.

## Regularization Hyperparameters

To avoid overfitting, several hyperparameters should be set.
- maximum depth of the tree (max_depth)
- maximum number of features evaluated for splitting at each node (max_features)
- maximum number of leaf nodes (max_leaf_nodes)
- minimum number of samples a node must have before it can be split (min_samples_split)
- minimum number of samples a node must have before it can be split, but expressed as a fraction of total number of weighted instances (min_weight_fraction_leaf)

In [ ]:
from sklearn.datasets import make_moons
X_moons, y_moons = make_moons (n_samples = 150, noise = 0.2, random_state = 42)

tree_clf1 = DecisionTreeClassifier(random_state=42)
tree_clf2 = DecisionTreeClassifier(min_samples_leaf=5, random_state=42)
tree_clf1.fit(X_moons, y_moons)
tree_clf2.fit(X_moons, y_moons)

In [ ]:
# extra code – this cell generates and saves Figure 6–3

def plot_decision_boundary(clf, X, y, axes, cmap):
    x1, x2 = np.meshgrid(np.linspace(axes[0], axes[1], 100),
                         np.linspace(axes[2], axes[3], 100))
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=cmap)
    plt.contour(x1, x2, y_pred, cmap="Greys", alpha=0.8)
    colors = {"Wistia": ["#78785c", "#c47b27"], "Pastel1": ["red", "blue"]}
    markers = ("o", "^")
    for idx in (0, 1):
        plt.plot(X[:, 0][y == idx], X[:, 1][y == idx],
                 color=colors[cmap][idx], marker=markers[idx], linestyle="none")
    plt.axis(axes)
    plt.xlabel(r"$x_1$")
    plt.ylabel(r"$x_2$", rotation=0)

fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)
plt.sca(axes[0])
plot_decision_boundary(tree_clf1, X_moons, y_moons,
                       axes=[-1.5, 2.4, -1, 1.5], cmap="Wistia")
plt.title("No restrictions")
plt.sca(axes[1])
plot_decision_boundary(tree_clf2, X_moons, y_moons,
                       axes=[-1.5, 2.4, -1, 1.5], cmap="Wistia")
plt.title(f"min_samples_leaf = {tree_clf2.min_samples_leaf}")
plt.ylabel("")
save_fig("min_samples_leaf_plot")
plt.show()

The figure on the right (regularized) will most likely generalize better, compared to the figure on the left which seems like it is overfitting.

In [ ]:
X_moons_test, y_moons_test = make_moons(n_samples=1000, noise=0.2,
                                        random_state=43)
tree_clf1.score(X_moons_test, y_moons_test)

In [ ]:
tree_clf2.score(X_moons_test, y_moons_test)

# Regression

In [ ]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

np.random.seed(42)
X_quad = np.random.rand(200,1) - 0.5
y_quad = X_quad ** 2 + 0.025 * np.random.randn(200,1)

tree_reg = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg.fit(X_quad, y_quad)

In [ ]:
# extra code – we've already seen how to use export_graphviz()
export_graphviz(
    tree_reg,
    out_file=str(IMAGES_PATH / "regression_tree.dot"),
    feature_names=["x1"],
    rounded=True,
    filled=True
)
Source.from_file(IMAGES_PATH / "regression_tree.dot")

In [ ]:
tree_reg2 = DecisionTreeRegressor(max_depth=3, random_state=42)
tree_reg2.fit(X_quad, y_quad)

In [ ]:
tree_reg.tree_.threshold

In [ ]:
tree_reg2.tree_.threshold

In [ ]:
# extra code – this cell generates and saves Figure 6–5

def plot_regression_predictions(tree_reg, X, y, axes=[-0.5, 0.5, -0.05, 0.25]):
    x1 = np.linspace(axes[0], axes[1], 500).reshape(-1, 1)
    y_pred = tree_reg.predict(x1)
    plt.axis(axes)
    plt.xlabel("$x_1$")
    plt.plot(X, y, "b.")
    plt.plot(x1, y_pred, "r.-", linewidth=2, label=r"$\hat{y}$")

fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)
plt.sca(axes[0])
plot_regression_predictions(tree_reg, X_quad, y_quad)

th0, th1a, th1b = tree_reg.tree_.threshold[[0, 1, 4]]
for split, style in ((th0, "k-"), (th1a, "k--"), (th1b, "k--")):
    plt.plot([split, split], [-0.05, 0.25], style, linewidth=2)
plt.text(th0, 0.16, "Depth=0", fontsize=15)
plt.text(th1a + 0.01, -0.01, "Depth=1", horizontalalignment="center", fontsize=13)
plt.text(th1b + 0.01, -0.01, "Depth=1", fontsize=13)
plt.ylabel("$y$", rotation=0)
plt.legend(loc="upper center", fontsize=16)
plt.title("max_depth=2")

plt.sca(axes[1])
th2s = tree_reg2.tree_.threshold[[2, 5, 9, 12]]
plot_regression_predictions(tree_reg2, X_quad, y_quad)
for split, style in ((th0, "k-"), (th1a, "k--"), (th1b, "k--")):
    plt.plot([split, split], [-0.05, 0.25], style, linewidth=2)
for split in th2s:
    plt.plot([split, split], [-0.05, 0.25], "k:", linewidth=1)
plt.text(th2s[2] + 0.01, 0.15, "Depth=2", fontsize=13)
plt.title("max_depth=3")

save_fig("tree_regression_plot")
plt.show()

Note that under each region, the predicted value will be the average target value of the instances in the region (since that will be the value that minimizes the sum of squared errors).

In [ ]:
# extra code – this cell generates and saves Figure 6–6

tree_reg1 = DecisionTreeRegressor(random_state=42)
tree_reg2 = DecisionTreeRegressor(random_state=42, min_samples_leaf=10)
tree_reg1.fit(X_quad, y_quad)
tree_reg2.fit(X_quad, y_quad)

x1 = np.linspace(-0.5, 0.5, 500).reshape(-1, 1)
y_pred1 = tree_reg1.predict(x1)
y_pred2 = tree_reg2.predict(x1)

fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)

plt.sca(axes[0])
plt.plot(X_quad, y_quad, "b.")
plt.plot(x1, y_pred1, "r.-", linewidth=2, label=r"$\hat{y}$")
plt.axis([-0.5, 0.5, -0.05, 0.25])
plt.xlabel("$x_1$")
plt.ylabel("$y$", rotation=0)
plt.legend(loc="upper center")
plt.title("No restrictions")

plt.sca(axes[1])
plt.plot(X_quad, y_quad, "b.")
plt.plot(x1, y_pred2, "r.-", linewidth=2, label=r"$\hat{y}$")
plt.axis([-0.5, 0.5, -0.05, 0.25])
plt.xlabel("$x_1$")
plt.title(f"min_samples_leaf={tree_reg2.min_samples_leaf}")

save_fig("tree_regression_regularization_plot")
plt.show()

Note that regression trees are prone to overfit if there are no regularization!

# Sensitivity to Axis Orientation

Note that decisiont trees are only able to make boundaries using orthogonal boundaries, and hence rotating the axis of data can easily influence the decision boundary for the data.

Hence, we can limit this by scaling the data, and then apply the principal component analysis transformation.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

pca_pipeline = make_pipeline(StandardScaler(), PCA())
X_iris_rotated = pca_pipeline.fit_transform(X_iris)
tree_clf_pca = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf_pca.fit(X_iris_rotated, y_iris)

In [ ]:
# extra code – this cell generates and saves Figure 6–8

plt.figure(figsize=(8, 4))

axes = [-2.2, 2.4, -0.6, 0.7]
z0s, z1s = np.meshgrid(np.linspace(axes[0], axes[1], 100),
                       np.linspace(axes[2], axes[3], 100))
X_iris_pca_all = np.c_[z0s.ravel(), z1s.ravel()]
y_pred = tree_clf_pca.predict(X_iris_pca_all).reshape(z0s.shape)

plt.contourf(z0s, z1s, y_pred, alpha=0.3, cmap=custom_cmap)
for idx, (name, style) in enumerate(zip(iris.target_names, ("yo", "bs", "g^"))):
    plt.plot(X_iris_rotated[:, 0][y_iris == idx],
             X_iris_rotated[:, 1][y_iris == idx],
             style, label=f"Iris {name}")

plt.xlabel("$z_1$")
plt.ylabel("$z_2$", rotation=0)
th1, th2 = tree_clf_pca.tree_.threshold[[0, 2]]
plt.plot([th1, th1], axes[2:], "k-", linewidth=2)
plt.plot([th2, th2], axes[2:], "k--", linewidth=2)
plt.text(th1 - 0.01, axes[2] + 0.05, "Depth=0",
         horizontalalignment="right", fontsize=15)
plt.text(th2 - 0.01, axes[2] + 0.05, "Depth=1",
         horizontalalignment="right", fontsize=13)
plt.axis(axes)
plt.legend(loc=(0.32, 0.67))
save_fig("pca_preprocessing_plot")

plt.show()

# High Variance of Decision Trees

Note that due to the nature of how the decision tree algorithm is used by Scikit-learn, retraining the same decision tree on the exact same data may produce very different models. hence, this motivates the prediction method by averaging over many tress to significantly reduce variance (e.g. random forest, boosting, etc).

# Exercises

## 1. What is the approximate depth of a decision tree trained (without restrictions) on a training set with one million instances?

$log_2$(1,000,000) = 19.9 -> 20 depth

## 2. Is a node’s Gini impurity generally lower or higher than its parent’s? Is it generally lower/higher, or always lower/higher?

A node's gini impurity should generally be lower than its parents as the decision tree will try and classify towards 1 class as much as possible. Generally it is lower, but it might not necessarily be ALWAYS higher.

## 3. If a decision tree is overfitting the training set, is it a good idea to try decreasing max_depth?

TRUE, a decision tree that is overfitted wil mean that the depth of the tree is too high, hence decreasing max_depth can be a good idea to overcome this issue.

## 4. If a decision tree is underfitting the training set, is it a good idea to try scaling the input features?

The scale of the features will not matter, the relative order / rank will matter.

## 5. If it takes one hour to train a decision tree on a training set containing one million instances, roughly how much time will it take to train another decision tree on a training set containing ten million instances? Hint: consider the CART algorithm’s computational complexity.

The computational complexity of training a Decision Tree is _O_(_n_ × _m_ log₂(_m_)). So if you multiply the training set size by 10, the training time will be multiplied by _K_ = (_n_ × 10 _m_ × log₂(10 _m_)) / (_n_ × _m_ × log₂(_m_)) = 10 × log₂(10 _m_) / log₂(_m_).


If _m_ = 10<sup>6</sup>, then _K_ ≈ 11.7, so you can expect the training time to be roughly 11.7 hours.

## 6. If it takes one hour to train a decision tree on a given training set, roughly how much time will it take if you double the number of features?

If the number of features double, then the time it takes to train the new decision tree will be double.

## 7. Train and fine-tune a decision tree for the moons dataset by following these steps:
1. Use make_moons(n_samples=10000, noise=0.4) to generate a moons dataset.
2. Use train_test_split() to split the dataset into a training set and a test set.
3. Use grid search with cross-validation (with the help of the GridSearchCV class) to find good hyperparameter values for a DecisionTreeClassifier. Hint: try various values for max_leaf_nodes.
4. Train it on the full training set using these hyperparameters, and measure your model’s performance on the test set. You should get roughly 85% to 87% accuracy.

In [ ]:
from sklearn.datasets import make_moons
X_moons, y_moons = make_moons (n_samples = 10000, noise = 0.4, random_state = 42)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_moons, y_moons, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import GridSearchCV

param = {
    "max_leaf_nodes": list(range(10,100)),
}

grid_search_dt = GridSearchCV(DecisionTreeClassifier(max_leaf_nodes= 10, random_state=42),
                         param,
                         cv=10,
                         scoring="roc_auc"
)
grid_search_dt.fit(X_train, y_train)

In [ ]:
y_pred = grid_search_dt.best_estimator_.predict(X_test)

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

In [ ]:
# extra code – this cell generates and saves Figure 6–3

def plot_decision_boundary(clf, X, y, axes, cmap):
    x1, x2 = np.meshgrid(np.linspace(axes[0], axes[1], 100),
                         np.linspace(axes[2], axes[3], 100))
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=cmap)
    plt.contour(x1, x2, y_pred, cmap="Greys", alpha=0.8)
    colors = {"Wistia": ["#78785c", "#c47b27"], "Pastel1": ["red", "blue"]}
    markers = ("o", "^")
    for idx in (0, 1):
        plt.plot(X[:, 0][y == idx], X[:, 1][y == idx],
                 color=colors[cmap][idx], marker=markers[idx], linestyle="none")
    plt.axis(axes)
    plt.xlabel(r"$x_1$")
    plt.ylabel(r"$x_2$", rotation=0)

# fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)
# plt.sca(axes[0])
plot_decision_boundary(grid_search_dt.best_estimator_, X_test, y_test,
                       axes=[-1.5, 2.4, -1, 1.5], cmap="Wistia")
plt.title("Decision Tree Boundary based on Testing Dataset")
# plt.sca(axes[1])
# plot_decision_boundary(tree_clf2, X_moons, y_moons,
#                        axes=[-1.5, 2.4, -1, 1.5], cmap="Wistia")
# plt.title(f"min_samples_leaf = {tree_clf2.min_samples_leaf}")
# plt.ylabel("")
# save_fig("min_samples_leaf_plot")
plt.show()

## 8. Grow a forest by following these steps:
1. Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit-Learn’s ShuffleSplit class for this.
2. Train one decision tree on each subset, using the best hyperparameter values found in the previous exercise. Evaluate these 1,000 decision trees on the test set. Since they were trained on smaller sets, these decision trees will likely perform worse than the first decision tree, achieving only about 80% accuracy.
3. Now comes the magic. For each test set instance, generate the predictions of the 1,000 decision trees, and keep only the most frequent prediction (you can use SciPy’s mode() function for this). This approach gives you majority-vote predictions over the test set.
4. Evaluate these predictions on the test set: you should obtain a slightly higher accuracy than your first model (about 0.5 to 1.5% higher). Congratulations, you have trained a random forest classifier!

In [ ]:
# Generate 1,000 subsets each containing 100 instances (0.0125 of the training instance)
from sklearn.model_selection import ShuffleSplit

rs = ShuffleSplit(n_splits=1000, train_size = 0.0125, random_state=42)

In [ ]:
# train a decision tree on each subset using the max_leaf_nodes parameter obtained previously
acc_list = []

# create an arbitrary valued array with size 1000, len(X_test)
# this is where we will store our prediction for each of the X_test data
Y_pred = np.empty([1000, len(X_test)], dtype=np.uint8)

for i, (train_index, test_index) in enumerate(rs.split(X_train)):
    tree_clf = DecisionTreeClassifier(max_leaf_nodes=grid_search_dt.best_estimator_.get_params()["max_leaf_nodes"])
    tree_clf.fit(X_train[train_index], y_train[train_index])
    y_pred = tree_clf.predict(X_test)
    acc_score = accuracy_score(y_test, y_pred)
    acc_list.append(acc_score)
    # set Y_pred value as y_pred
    Y_pred[i] = y_pred
    

In [ ]:
# Compute the mean of the accuracy scores in acc_list, indeed it is around 80% accuracy
np.mean(acc_list)

In [ ]:
from scipy.stats import mode
# compute the most frqeuently appearing predicted class, note the axis = 0 means across all the rows
# aka mode for each column
# the first value is the majority vote value, the second value is the total count of said value
y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

In [ ]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

Above we can see that indeed by using 1000 trees, we can have similar level of accuracy when we only have a limited amount of data!